<a href="https://colab.research.google.com/github/jrhumberto/cd/blob/main/005_NLP_NEO4J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fonte:

- https://towardsdatascience.com/construct-the-matrix-interaction-network-based-on-the-movie-script-738b4fa9b46d

- https://github.com/tomasonjo/blogs/blob/master/matrix/MatrixNLP.ipynb



In [1]:
!sudo apt install tesseract-ocr
!sudo apt-get install poppler-utils 
# Install dependencies
!pip install -U selenium neo4j pytesseract pdf2image spacy --upgrade
!python -m spacy download en_core_web_sm

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,386 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [2]:
# Setup selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [691 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 http://security.ubuntu.co

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [3]:
wikifan_url = "https://matrix.fandom.com/wiki/Category:Characters_in_The_Matrix"

member_list = []
wd.get(wikifan_url)
members = wd.find_elements_by_class_name("category-page__member-link")
for m in members:
  member_list.append({'url':m.get_attribute('href'), 'name': m.text})

# Manually append Trinity
member_list.append({'url': 'https://matrix.fandom.com/wiki/Trinity', 'name': 'Trinity'})



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """


In [4]:
for m in member_list:
  wd.get(m['url'])
  elements = wd.find_elements_by_class_name("pi-data")
  for e in elements:
    try:
      label = e.find_element_by_tag_name("h3")
      value = e.find_element_by_tag_name("div")
      m[label.text] = value.text
    except:
      pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [5]:
from neo4j import GraphDatabase
# Change the host and user/password combination to your neo4j
# Will not work with a localhost bolt url
host = 'bolt://44.200.249.124:7687'
user = 'neo4j'
password = 'battle-manpower-sand'
driver = GraphDatabase.driver(host,auth=(user, password))



In [7]:
entity_query = """
UNWIND $data as row
CREATE (c:Character)
SET c += row
"""
with driver.session() as session:
    session.run(entity_query, {'data': member_list})



ServiceUnavailable: ignored

In [ ]:
import requests
import pdf2image
import pytesseract

pdf_link = "https://www.dailyscript.com/scripts/the_matrix.pdf"

pdf = requests.get(pdf_link)
doc = pdf2image.convert_from_bytes(pdf.content)

# Get the article text
article = []
for page_number, page_data in enumerate(doc):
    # First page is the title
    if page_number == 0:
      continue
    txt = pytesseract.image_to_string(page_data, lang='eng').encode("utf-8")
    article.append(txt.decode("utf-8"))

article_txt = " ".join(article)


In [ ]:
# a bit of cleaning
article_clean_txt = "\n".join([line for line in article_txt.split("\n") if not "THE MATRIX" in line and not "CONTINUED" in line])


In [ ]:
# Optionally store to file
with open('/matrix_script.txt', 'w') as writefile:
    writefile.write(article_clean_txt)

from google.colab import files
files.download('/matrix_script.txt')



In [ ]:
#Split by scenes
scenes = []
single_scene = []
for line in article_clean_txt.split("\n"):
  # If empty line
  if not line or line.startswith("OMITTED"):
    continue
  if line.startswith("INT.") or line.startswith("EXT.") or line.startswith("THE END"):
    scenes.append(("\n").join(single_scene))
    single_scene = []
  single_scene.append(line)

scene_names = [el.split("\n")[0] for el in scenes]



In [ ]:
def get_matcher_patterns(name):
    """Function that construct a SpaCy rule-based pattern from a name"""
    matcher_pattern = []
    clean_name = name.replace("The", "").strip()
    parts_of_name = clean_name.split(" ")
    # Append the capitalized version
    matcher_pattern.append([{"LOWER": n.lower(), "IS_TITLE": True} for n in parts_of_name])
    # Append the uppercased version
    matcher_pattern.append([{"LOWER": n.lower(), "IS_UPPER": True} for n in parts_of_name])
    return matcher_pattern



In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

for m in member_list:
    matcher.add(m['name'], get_matcher_patterns(m['name']))

In [ ]:
x = 0

for s, sn in zip(scenes, scene_names):
    characters = set()
    doc = nlp(s.replace("\n", " "))
    matches = matcher(doc)
  
    for match_id, start, end in matches:
        characters.add(str(doc[start:end]).lower())
  
    entity_query = """
        MERGE (s:Scene {id: $scene_id})
        SET s.title = $scene_title
        WITH s
        UNWIND $characters as char
        MATCH (c:Character)
        WHERE toLower(c.name) = CASE WHEN NOT char IN ["oracle", "priestess"] THEN char ELSE "the " + char END
        MERGE (c)-[:IN_SCENE]->(s)
        """
    with driver.session() as session:
        session.run(entity_query, {'scene_id': x, 'scene_title': sn, 'characters': list(characters)})

    x += 1

In [ ]:
import pandas as pd

def read_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())



In [ ]:
read_query("""
    MATCH (n:Character) WHERE NOT (n)--()
    RETURN n.name
""")

In [ ]:
read_query("""
    MATCH (n:Character)
    RETURN n.name AS character,
       size((n)-[:IN_SCENE]->()) as scenes
    ORDER BY scenes DESC
    LIMIT 5
""")

In [ ]:
read_query("""
    MATCH (n1:Character)-[:IN_SCENE]->()<-[:IN_SCENE]-(n2:Character)
    WHERE id(n1) < id(n2)
    RETURN n1.name AS character1,
       n2.name AS character2,
       count(*) AS count
    ORDER BY count DESC
    LIMIT 5
""")

In [ ]:
read_query("""
    MATCH (n1:Character)-[:IN_SCENE]->()<-[:IN_SCENE]-(n2:Character)
    WHERE id(n1) < id(n2)
    WITH n1, n2, count(*) AS count
    MERGE (n1)-[r:INTERACTS]-(n2)
    SET r.weight = count
""")

In [ ]:
read_query("""
CALL gds.graph.create("matrix", "Character", {INTERACTS: {orientation:"UNDIRECTED", properties:"weight"}})
""")

In [ ]:
read_query("""
CALL gds.pageRank.write("matrix", {relationshipWeightProperty:"weight", writeProperty:"pagerank"})
""")

In [ ]:
read_query("""
CALL gds.louvain.write("matrix", {relationshipWeightProperty:"weight", writeProperty:"louvain"})
""")